In [22]:
import os, json
import pandas as pd
from tqdm.notebook import tqdm

files_list = os.listdir('/data/brief')

data = []
for file in tqdm(files_list):
    if not file.endswith('.json'): continue
    with open('/data/brief/'+file, 'r') as fin:
        item = json.load(fin)
    item.update(json.loads(item['json_solution'])['SolutionInfo'])
    data.append(item)
    
df = pd.DataFrame(data)
len(df), df.columns

  0%|          | 0/720 [00:00<?, ?it/s]

(720,
 Index(['gurobi_logs', 'json_solution', 'time', 'gurobi_runtime', 'status_code',
        'status', 'mip_gap', 'gurobi_model_fingerprint', 'binary_variables',
        'solutions_count', 'node_count', 'simplex_iterations_count',
        'time_limit', 'threads', 'coverage_density', 'communication_density',
        'instance_key', 'gurobi_model_type', 'K', 'M', 'seed', 'pois',
        'sensors', 'sinks', 'area', 'coverage', 'communication', 'queued',
        'model', 'Status', 'Runtime', 'ObjVal', 'ObjBound', 'ObjBoundC',
        'MIPGap', 'IterCount', 'BarIterCount', 'NodeCount', 'SolCount',
        'IntVio', 'BoundVio', 'ConstrVio', 'PoolObjBound', 'PoolObjVal'],
       dtype='object'))

In [23]:
show = df[[# 'instance_key',
    'pois', 'sensors', 'area', 'coverage', 'communication', 'seed',
    'K', 'M', 'model',
    
    'binary_variables',
    
    'status', 'solutions_count', 'node_count',
    'gurobi_runtime', 'simplex_iterations_count',
    
    
    # 'Status', 'SolCount', 'NodeCount',
    # 'Runtime', 'IterCount', 'BarIterCount',
    'MIPGap', 'ObjVal', 'ObjBound', 'ObjBoundC', 
    # 'IntVio', 'BoundVio', 'ConstrVio', 'PoolObjBound', 'PoolObjVal'
   ]].sort_values([
    'pois', 'sensors', 'area', 'coverage', 'communication', 'seed',
    'K', 'M', 'model',
]).reset_index(drop=True)

len(show), show.columns

(720,
 Index(['pois', 'sensors', 'area', 'coverage', 'communication', 'seed', 'K',
        'M', 'model', 'binary_variables', 'status', 'solutions_count',
        'node_count', 'gurobi_runtime', 'simplex_iterations_count', 'MIPGap',
        'ObjVal', 'ObjBound', 'ObjBoundC'],
       dtype='object'))

In [24]:
show.to_csv('/data/brief.csv', index=None, sep=';')

In [4]:
! pip install plotly

     |████████████████████████████████| 27.7 MB 18.0 MB/s eta 0:00:01    |███████████████▋                | 13.5 MB 7.9 MB/s eta 0:00:02
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [17]:
import plotly.express as px
from numpy import log10

show.loc[:, 'log_gurobi_runtime'] = show['gurobi_runtime'].apply(log10)
show.loc[:, 'log_binary_variables'] = show['binary_variables'].apply(log10)
show.loc[:, 'km_model'] = show['model']+' '+show['K'].astype(str)+','+show['M'].astype(str)

fig = px.scatter_3d(show, x='log_binary_variables', y='M', z='log_gurobi_runtime',
                    color='model')
fig.show()

In [18]:
fig = px.scatter_3d(show, x='log_binary_variables', y='M', z='log_gurobi_runtime',
                    color='sensors')
fig.show()

In [19]:
fig = px.scatter_3d(show, x='log_binary_variables', y='M', z='log_gurobi_runtime',
                    color='pois')
fig.show()

In [20]:
fig = px.scatter_3d(show, x='sensors', y='pois', z='log_gurobi_runtime',
                    color='km_model')
fig.show()